# Tutorial 1: Creating the framework's input files
The EGM framework require multiple input files. You should refer to the `Simulaciones-WorkFlow.docx` for a deep description of the hydrodynamic model input files.


The framework also require some external files to apply some special features, but those will be explained here in this tutorial.


## STEP 0: Load the egmlib library
In the code below replace the string `lib_folder` with the path to folder where the **egmlib.py** file is and than run the cell. Just note that we should use `/` or `\\` instead of a single `\` as Python understand this last as a indicator of a special character.


The egmlib library already include the import of many other libraries, as **numpy** and **matplotlib** that will be used later in this notebook.

In [ ]:
lib_folder = 'C:/Wetlands/Tutorials'
from sys import path
path.append(lib_folder)
from egmlib import *

***
Before we move on, this tutorial covers only some applications of the **EGMproject** class in the **egmlib** library. The attributes and methods of this class mimic some features found in *Simulaciones* software, when creating a new project. However, the main purpose of this class is to quickly create a rectangular domain with basic components, similarly to those used in the study of effect of structures in SE Australia coastal wetlands.
***

## STEP 1: Setting domain properties
The models within the EGM framework strongly relly on cells and links characteristics to carry on their computations. Inside the **egmlib** library, we have a particular Python class called **EGMproject**, which reunite properties and methods/functions for creation of simple rectangular domains. In this first step, we will use many functions of this class to create a simulation domain with an inner channel, an embankment and a culvert.
<br></br>
### 1.1: Initial domain setup
Define the number of rows and columns in the domain, create and initialise an **EGMproject** object, and then define the values for *x* and *y* coordinates, as well the dimensions *dx* and *dy* of each domain-cell.
<br></br>
We will also update the attribute *Mann*, which stores the Manning's roughness coefficient in each cell, to *n = 0.30* over the entire domain.

In [ ]:
# Creating a variable from EGMproject class
dom = EGMproject()

# Initialise its properties based on a given number of rows and columns
dom.initialize_from_dimensions(nrows=21, ncols=82)

#now 'dom' have some attributes. Let's see how many cells and links are on it
print('Number of cells', dom.NoC)
print('Number of links', dom.NoL)

# Set dx = dy = 10 m to all cells in the domain. These attributes are arrays with (nrows,ncols) shape
dom.dx[:,:] = 10.
dom.dy[:,:] = 10.

# Update the cells' center coordinates using the bottom-left corner of the bottom-left cell as the
#system's origin (0,0).
dom.set_coordinates()
print('\nLets check a sample of x and y matrices')
display(dom.x)
display(dom.y)

# Update Manning's roughness coefficient to n = 0.30 at all cells
dom.Mann[:,:] = 0.30

### 1.2: Special Tiles
The EGM Framework "understands" that some set of cells should be treated separately from the rest of the domain. Cells listed in these areas can be excluded before running the EGM models (Vegetation, Accretion and Hillslope Diffusion), so they will maintain their parameters through the entire simulation. <br></br> 
It is necessary to define at least one selection as "TIDE" or "INPUT_CELLS", because the Sediment Transport Model will set the sediment concentration in these areas as the same concentration provided by the sediment-input-series.

In [ ]:
# It will be easy to select the areas if we can see where the cells are in the domain.
#For such, we will take advantage of pandas.DataFrame.to_html() function. But before we need to copy
#the numbers matrix to a pandas DataFrame
from IPython.display import HTML
numdf = pd.DataFrame(dom.number, columns=dom.x[0,:], index=dom.y[:,0])
display(HTML(numdf.to_html()))

In this tutorial we will create two selections: "TIDE" and "RIVER", to indicate which cells are part of the tide-input area and which ones are part of an inner channel in the floodplain.
<br></br>
<br></br>
To do so, we will use the EGMproject function **set_special_tiles(** *name*, *list_of_cells* **)** to create a new attribute that will store this information. Actually, this function can receive a third parameter, *mann*, to set the Manning's roughness coefficient in the special area. For now we will maintain its default value of 0.035. 
<br></br>
<br></br>
We will also review the coordinate system to change its origin to the bottom-left corner of the first floodplain column, i.e. the "TIDE" area will be in the negative x-sector. This will be very usefull later when plotting information from the floodplain only.
<br></br>
<br></br>
Another change to be made is to make the "RIVER" area thinner by changing the *dy* attribute to 5 m.

In [ ]:
# The first special area will be the first two columns, which we will name as "TIDE".
cells = dom.number[:,0:2]
dom.set_special_tiles("TIDE", cells)
#we can access a new attribute now: dom.tiles["TIDE"]["cells"], which will return an array with the cells' numbers
#that should be treated as "TIDE" area

# The second area is the bottom row, as we will change it to work like a channel-type cell (although for the
#hydrodynamic model it will still be a land-type one)
cells = dom.number[-1,2:]
dom.set_special_tiles("RIVER", cells)

print('Special areas in the domain:')
display("TIDE:", dom.tiles["TIDE"]["cells"])
display("RIVER:", dom.tiles["RIVER"]["cells"])

# Re-setting the coordinate system: 1) Change dy for RIVER-row; 2) set x-coordinates to start at -2*dx, as
#we selected the first two columns as TIDE area
dom.dy[-1,:] = 5.
dom.set_coordinates(xWest=-20)

# Print the cells' number table again
numdf = pd.DataFrame(dom.number, columns=dom.x[0,:], index=dom.y[:,0])
print("New coordinates' system:")
display(HTML(numdf.to_html()))

## 1.3 Defining elevation
When an instance of **EGMproject** is created, the bottom elevation matrix is initialised with all values set to 0 m.

In this tutorial we will create a ramp in the x-direction with a slope of 0.001 m/m. Later, we will drop the elevation in the RIVER-row by 0.4 m

In [ ]:
dom.z = dom.x * 0.001
dom.z[dom.tiles["RIVER"]["mask"]] -= 0.4    #not mentioned before, but when .set_special_tiles is called it also
#includes a mask matrix to access that cells in the (nrows,ncols) matrices
print('Elevation in floodplain downstream border:', dom.z[0,2])
print('Elevation in floodplain upstream border:', dom.z[0,-1])

## 1.4: Adding embankment
Placing an embankment in the domain is the same of removing the links between those cells separated by such structure.


The EGMproject class provide two ways to insert an embankment in the domain. In the first one it is necessary to create a list with pairs of cells' number where the embankment will cut the connection between them. The function **remove_links(** *list_of_pairs_of_cells* **)** is used to carry on this task. The second method invokes the function **place_embankment_at_x(** *x*, *selected_rows* **)** and/or **place_embankment_at_y(** *y*, *selected_cols* **)** to automatically search those pairs of cells in the surroundings of the given coordinate (*x* or *y*) in the given subspace (*selected_rows* or *selected_cols*) .


In this tutorial, we will place an embankment in the middle of the floodplain, at x = 400 m, crossing the floodplain in the North-South direction. We need to mind to not include the bottom row, which will work as the inner channel.

In [ ]:
# To place an embankment at x=400 m, we need will use the function place_embankment_at_x(), passing as arguments
#x=400 and a list of rows that have a dy value higher than 9.99 m, i.e. those rows which dy was not reduced to 5 m.
non_river_rows = [i for i in range(dom.nrows) if dom.dy[i,0] > 9.99]
pairs = dom.place_embankment_at_x(400, non_river_rows)
print('Pairs of cells:')
display(pairs)

## 1.5 Adding culvert
Adding a culvert is a simple operation. One needs to provide the number of the linked cells where the culvert will be installed, and its parameters as well.


We will use the function **set_culvert()**, which needs to receive at least the following parameters:
* *cell1* = cell's number of the first cell in the link
* *cell2* = cell's number of the second cell in the link
* *base_width* = 1st step culvert's width
* *gate_opening* = heigh between the bottom elevation of the 1st step and the gate level


All other culvert's parameters can be defined from the domain elevation at the linked cells, but the discharge coefficient, which default value is 0.8.


Let's add a culvert aligned with embankment, but at the inner channel row.

In [ ]:
link = dom.set_culvert(cell1=1681, cell2=1682, base_width=0.8, gate_opening=0.5)
print("Culvert's parameters:")
display(dom.params[link,:])

## 1.6 Mapping domain
This step is not really necessary to compose any of the input files. However, it is good to have a look if all changes made so far took place correctly.


For such, we will use the function **map_domain()** which use the current information in the **EGMproject**'s variable to draw a colormap of the elevation, overlapped by a scatter plot of the position of cell's centres and links. The function returns a matplotlib.pyplot's figure and axis. Thus, one can add extra features to the composition before drawing the figure with a pyplot's savefig() or show() function.

In [ ]:
# Creating the base map of domain's terrain and cell/links location
f, a = dom.map_domain()

# Add the embankment as a thick black line
a.plot([400., 400.], [205., 5.], 'k-', lw=3)
pyplot.show()
pyplot.close(f)

## 1.7 Profiles for Hillslope Diffusion (HD) Model
To apply the HD model in a EGM simulation using the current domain, it is necessary to provide the sequences of cells that define each hillslope profile to be smoothed by the HD model.


For this current domain, we would like to apply the HD model throught the x and y axis. We could just take the sequences from each row and each column through the self.number matrix. However, we need to mind to not include cells in the special areas (TIDE and RIVER), and to stop the profile at the embankment.


The good news is that the function **build_hillslope_profiles()** can sweep through the land-to-land links in the floodplain area to find these profiles for us automatically!


**Attention please:** the function build_hillslope_profiles() returns only the list with sequence of cells. However, to create the file with this information it is also necessary to provide the distance between the cells on each profile. Here we will use the default value of 10 m. But if you will create a domain with different cell size, than you must address this when the hillslope_profile_write() function is called

In [ ]:
# Finding horizontal and vertical continuos line
dom.build_hillslope_profiles()

xp, yp = dom.x.flatten(), dom.y.flatten()
f, a = pyplot.subplots(figsize=(9,4), dpi=100, constrained_layout=True)
for p in dom.profiles:
    a.plot(xp[p], yp[p], 'k-', lw=1)
a.plot(xp, yp, 'b.')
a.set_ylabel('y [m]')
a.set_xlabel('x [m]')
pyplot.show()
pyplot.close(f)

# 2 Boundary Conditions
So far we have worked with domain-related features. Now we need to mind the water input series that will force the hydrodynamic run. There are at least two more steps to run: set the input links, set the input series.


# 2.1 Set Input Links
Although we already defined an special set of cells as "TIDE" area, we need to list the pairs of linked cells where the water level will be defined by the input series. In our case here, we want to set all the pairs between column 0 and column 1 as boundary condition

In [ ]:
# HxT boundary locations are composed by [first/inlet cell number, second/outlet cell number, status: 0=inactive;
#1=active]. It must be converted to np.ndarray because of the boundary file writing function
dom.bound_at = np.array([[dom.number[row,0], dom.number[row,1], 1] for row in range(dom.nrows)], dtype=int)
display(dom.bound_at)

## 2.2 Input Series
To run the hydrodynamic model we need to provide an initial series of water *depths* on each boundary link. However, for the EGM simulation it is necessary water *levels* and sediment concentration for each EGM time step. Let's take care of each one separately.


### 2.2.1 Initial water-depths series
The series of water depths attached to each boundary-link must have the same time-length and time-step. We will first create a water *level* series using a sinusoidal formula with the following characteristics:
* wave-length = 12 hours
* tidal range = 1 m
* mean level = 0 m
* time-step = 300 s
* series time-length = 48 hours

This single water level series will supply the EGMproject function **set_boundary_depths()** which will use the terrain elevation at the first cell of each boundary-link to transform *level* in *depth*, thus creating a new attribute with the matrix of water depths on each boundary-link. This matrix will have a shape (*n*,*m*) where *n* is the number of records in the series and *m* the number of boundary-links


One last thing. We need to manually create/update the attribute *bound_time_step* with the time step used to create this series.

In [ ]:
# Calling the sinusoidal wave creator function
time_stamps, levels = sinusoid(amplitude=1.0, mean_level=0.0, wave_length=12*3600, time_step=300,
                               final_time=48*3600)
dom.bound_time_step = 300

# Set water depths at boundary-links
dom.set_boundary_depths(levels)
pyplot.plot(time_stamps/3600, dom.bound_h[:,0], label='input depths')

# Just to compare, let's plot the sinusoidal levels and the water depths at the first boundary-link
pyplot.plot(time_stamps/3600, levels, label='levels')
pyplot.xlabel('hour')
pyplot.ylabel('metres')
pyplot.legend()
pyplot.grid()
pyplot.show()

### 2.2.2 EGM water-level series
As in the EGM simulation we usually run the framework over a sequence of years, we must provide the equivalent water level series - to be used as input in a similar **set_boundary_depths()** function - for each year.

Therefore we need to:
1. Define a sequence of years to obtain the rise in the MSL from the RCPP 8.5 scenario
2. Create a default/initial water level series which will be copied and updated with the sea-level-rise on each year
3. Create a pandas.DataFrame to store all series, using the years as column names and the time stamp of each record as index.

In [ ]:
# Years for the EGM simulation
years = np.arange(2000, 2101, 20)    #Every 20 years between 2000 and 2100

# Accumulated SLR for each year
slr = SLR(years)

# Base sinusoidal wave (longer than previous one set for the initial boundary condition)
time_stamps, levels = sinusoid(amplitude=1.0, final_time=4*86400)

# Matrix of water levels on each year, added the SLR
wl = np.transpose(np.tile(levels, (slr.size,1)))
wl = np.add(wl, slr)
inp_levels = pd.DataFrame(wl, columns=years, index=time_stamps)

### 2.2.3 EGM sediment concentration series
For each record in the EGM water level series, there must be a record of sediment concentration (g/m3). Here we will create flat series with the same value at all time and years.

In [ ]:
# Create a copy of EGM water level input series to use the same column and index names
ssc = inp_levels.copy()
ssc.iloc[:,:] = 40    # Replace all data by a constant value of 40 g/m3

# Plot EGM inputs
inp_levels.plot()
ssc.plot()

## 2.3 Run-time settings and initial condition
Originally the hydrodynamic simulation splits the simulation period into 4 intervals. Thus, each one has its own solution’s time-step and its final time.

To finish the EGM project we need to defined the following run-time parameters:
* hydrodynamic solution time step (usually 1s) for each interval
* interval's final time (usually low values for the first 3 intervals, and the last with the full-simulation final time)
* time step of output series (in seconds)
* matrix of initial depths

In [ ]:
# time-step used in the solution of hydrodynamic model
dom.solution_time_step = [1, 1, 1, 1]

# final time of the hydrodynamic/sediment transport simulation. Computed based on boundary depths series length
dom.hydrodynamic_final_time = [60, 120, 180, dom.bound_time_step * (dom.bound_h.shape[0] - 1)]

# time-step used to record outputs of hydrodynamic model (also used to solve the sediment transport model)
dom.output_time_step = 300

# The initial depth will be calculated assuming that the water LEVEL at t = 0 is the same in all domain. So, those
#cells under this level will have some water depth, and those above will be dry. We can define this level at t = 0
#manually, or using the first value in the array 'levels' returned by the sinusoid function, or recomputing it from
#the first record of the depth series at the first boundary link. This last one may sounds more complex, but it is
#the best option to maintain consistence between the boundary and initial depth series
lev0 = dom.bound_h[0,0] + dom.z.flatten()[dom.bound_at[0][0]] #water level = water depth + bottom elevation
print('Initial water level =', lev0)

# creating attribute initial_depth
dom.set_initial_depths(lev0)

# Step 3: Saving information to files
Time to dump all domain, boundary condition, initial condition and input series information in the files expected by the HST and EGM simulations! For such, create a folder named **domain01** where you will place all tutorials file.


Most of this task is carried on by the EGMproject's function **finalize(** *folder_name* **)**, where *folder_name* is the path where the files will be saved. However, the EGM input series stored in pandas.DataFrame need to be saved separately with the embed DataFrame function **to_csv(** *csv_file* **)**

In [ ]:
save_to_folder = 'C:/Wetlands/Tutorials/domain01'
dom.finalize(save_to_folder)

inp_levels.to_csv('C:/Wetlands/Tutorials/water_levels_1.csv')
ssc.to_csv('C:/Wetlands/Tutorials/sediment_conc_1.csv')

# THE END
All files are ready. However, remember that the hydrodynamic model requires other \*.dat files, which we don't need to change at all. You can just copy them from the *Common Files* folder.


Another example of domain creation can be found in the `example_domain2` notebook